In [ ]:
!pip install colorama
!pip install imblearn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from colorama import Fore, Back, Style
import warnings
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from plotly.offline import plot, iplot, init_notebook_mode
warnings.filterwarnings("ignore")
%matplotlib inline
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score
from imblearn.metrics import geometric_mean_score
from sklearn import svm
import six
import sys
sys.modules['sklearn.externals.six'] = six
from mlxtend.classifier import StackingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.datasets import load_iris

In [ ]:
nic= pd.read_csv('titanic.csv')
print(nic)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [ ]:

print(nic.info())
print(nic.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.48659

In [ ]:
print(nic.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [ ]:
nic = nic.dropna()
print(nic)

     PassengerId  Survived  Pclass  \
1              2         1       1   
3              4         1       1   
6              7         0       1   
10            11         1       3   
11            12         1       1   
..           ...       ...     ...   
871          872         1       1   
872          873         0       1   
879          880         1       1   
887          888         1       1   
889          890         1       1   

                                                  Name     Sex   Age  SibSp  \
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
6                              McCarthy, Mr. Timothy J    male  54.0      0   
10                     Sandstrom, Miss. Marguerite Rut  female   4.0      1   
11                            Bonnell, Miss. Elizabeth  female  58.0      0   
..                                                 ...     ...   ... 

In [ ]:
numeric_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
nic[numeric_features] = nic[numeric_features].fillna(nic[numeric_features].mean())

In [ ]:
X = nic.drop('Survived', axis=1)
y = nic['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
k_best = 5  # Adjust the number of features based on your analysis
numeric_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
categorical_features = ['Sex', 'Embarked']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

selector = SelectKBest(f_classif, k=k_best)

In [ ]:
model_pipeline_dt = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', selector),
    ('model', DecisionTreeClassifier(random_state=42))
])


In [ ]:
model_pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', selector),
    ('model', RandomForestClassifier(random_state=42))
])


In [ ]:
model_pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', selector),
    ('model', LogisticRegression(random_state=42))
])

In [ ]:
models = [
    ('dt', model_pipeline_dt),
    ('rf', model_pipeline_rf),
    ('lr', Pipeline([
        ('preprocessor', preprocessor),
        ('selector', selector),
        ('model', LogisticRegression(random_state=42, C=1.0, max_iter=1000))  # Adjust hyperparameters
    ]))
]

In [ ]:
stacking_classifier = StackingClassifier(estimators=models, final_estimator=LogisticRegression())

# Train and evaluate the stacking model
stacking_classifier.fit(X_train, y_train)
y_pred_stacking = stacking_classifier.predict(X_test)

# Evaluate the stacking model
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)
conf_matrix_stacking = confusion_matrix(y_test, y_pred_stacking)
classification_rep_stacking = classification_report(y_test, y_pred_stacking)

# Display results
print("Improved Stacking Model:")
print(f"Accuracy: {accuracy_stacking:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix_stacking)
print("\nClassification Report:")
print(classification_rep_stacking)

Improved Stacking Model:
Accuracy: 0.65

Confusion Matrix:
[[ 4 10]
 [ 3 20]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.29      0.38        14
           1       0.67      0.87      0.75        23

    accuracy                           0.65        37
   macro avg       0.62      0.58      0.57        37
weighted avg       0.63      0.65      0.61        37

